In [9]:
import sys
import pathlib
import polars as pl
import umap

sys.path.append("../../")
from utils.io_utils import load_profiles
from utils.data_utils import split_meta_and_features

In [10]:
# setting paths
results_dir = pathlib.Path("./results/cfret-screen/").resolve(strict=True)

# set cfret-screen data
cfret_data_path = (results_dir / "cfret_screen_treatment_clustered.parquet").resolve(
    strict=True
)

In [11]:
# load dataset and split meta and features
cfret_screen_df = load_profiles(
    cfret_data_path, convert_to_f32=True
)  # converted to f32 to save memory
cfret_screen_meta, cfret_screen_feats = split_meta_and_features(cfret_screen_df)
cfret_screen_df.head()

Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_cell_type,Metadata_heart_failure_type,Metadata_treatment,Metadata_Pathway,Metadata_Nuclei_Location_Center_X,Metadata_Nuclei_Location_Center_Y,Metadata_Cells_Location_Center_X,Metadata_Cells_Location_Center_Y,Metadata_Image_Count_Cells,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cells_Number_Object_Number,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,Metadata_Nuclei_Number_Object_Number,Metadata_Site,Metadata_cell_id,Cytoplasm_AreaShape_Area,Cytoplasm_AreaShape_MajorAxisLength,Cytoplasm_AreaShape_Zernike_4_0,Cytoplasm_AreaShape_Zernike_5_1,Cytoplasm_AreaShape_Zernike_6_0,Cytoplasm_AreaShape_Zernike_6_2,Cytoplasm_AreaShape_Zernike_7_1,Cytoplasm_AreaShape_Zernike_7_3,Cytoplasm_AreaShape_Zernike_8_0,Cytoplasm_AreaShape_Zernike_8_2,Cytoplasm_AreaShape_Zernike_9_1,Cytoplasm_AreaShape_Zernike_9_3,Cytoplasm_AreaShape_Zernike_9_5,Cytoplasm_AreaShape_Zernike_9_7,Cytoplasm_Correlation_Correlation_ER_Hoechst,Cytoplasm_Correlation_Correlation_ER_PM,…,Nuclei_Texture_Correlation_Hoechst_3_02_256,Nuclei_Texture_Correlation_Hoechst_3_03_256,Nuclei_Texture_Correlation_Mitochondria_3_00_256,Nuclei_Texture_Correlation_Mitochondria_3_01_256,Nuclei_Texture_Correlation_Mitochondria_3_02_256,Nuclei_Texture_Correlation_Mitochondria_3_03_256,Nuclei_Texture_Correlation_PM_3_00_256,Nuclei_Texture_Correlation_PM_3_01_256,Nuclei_Texture_Correlation_PM_3_02_256,Nuclei_Texture_Correlation_PM_3_03_256,Nuclei_Texture_DifferenceEntropy_Hoechst_3_00_256,Nuclei_Texture_DifferenceEntropy_Hoechst_3_02_256,Nuclei_Texture_InfoMeas1_ER_3_00_256,Nuclei_Texture_InfoMeas1_ER_3_01_256,Nuclei_Texture_InfoMeas1_ER_3_02_256,Nuclei_Texture_InfoMeas1_ER_3_03_256,Nuclei_Texture_InfoMeas1_PM_3_00_256,Nuclei_Texture_InfoMeas1_PM_3_01_256,Nuclei_Texture_InfoMeas1_PM_3_02_256,Nuclei_Texture_InfoMeas1_PM_3_03_256,Nuclei_Texture_InfoMeas2_PM_3_00_256,Nuclei_Texture_InfoMeas2_PM_3_01_256,Nuclei_Texture_InfoMeas2_PM_3_02_256,Nuclei_Texture_InfoMeas2_PM_3_03_256,Nuclei_Texture_InverseDifferenceMoment_Hoechst_3_00_256,Nuclei_Texture_InverseDifferenceMoment_Hoechst_3_01_256,Nuclei_Texture_InverseDifferenceMoment_Hoechst_3_02_256,Nuclei_Texture_InverseDifferenceMoment_Hoechst_3_03_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_00_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_01_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_02_256,Nuclei_Texture_InverseDifferenceMoment_PM_3_03_256,Nuclei_Texture_SumEntropy_PM_3_01_256,Metadata_cluster_id,Metadata_cluster_n_cells,Metadata_treatment_n_cells,Metadata_cluster_ratio
str,i64,i64,str,str,str,str,f32,f32,f32,f32,i64,i64,str,str,i64,i64,i64,i64,str,str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,cat,u32,u32,f32
"""B""",2,7,"""healthy""",null,"""DMSO_heart_11""",null,870.048157,222.975906,883.760315,261.616211,8,2,"""localhost240927060001""","""B02""",1,1,3,3,"""f07""","""12575616795011807720""",-0.751363,0.572923,-0.397076,0.280466,-0.842051,0.921933,-0.808205,-0.152162,-0.576562,1.018035,-0.555971,1.136591,-1.010685,-0.580809,0.296295,0.374481,…,0.776713,-0.060115,-0.47829,0.369701,0.664598,-0.595822,-0.779385,-1.10438,0.019679,-0.081576,0.899131,0.131613,0.288529,-0.396068,-1.475314,0.104475,0.605291,0.480656,-0.418191,0.05484,-0.245545,-0.194699,0.449148,0.153167,-1.314356,-0.527268,-0.28336,-0.966427,-0.028467,0.025132,0.531559,0.161083,-0.084311,"""DMSO_heart_11_louvain_3""",324,1720,18.83721
"""B""",2,7,"""healthy""",null,"""DMSO_heart_11""",null,372.665131,78.150612,422.940613,121.357254,9,3,"""localhost240927060001""","""B02""",1,1,3,3,"""f08""","""3793444334871218055""",-1.315906,1.653718,-0.660428,-1.684414,-0.408983,-0.805361,-1.386725,-1.901982,-0.170266,-0.830062,-1.194093,-1.405091,-1.373065,-1.294781,0.279446,0.891917,…,0.260714,-0.725359,0.799276,1.3109,0.532934,0.074106,0.416485,

In [12]:
# transforming cfret-screen data with UMAP
umap_model = umap.UMAP(n_components=2, random_state=0, n_jobs=1)

# Remove duplicate indexing: cfret_screen_feats[cfret_screen_feats]
umap_embeddings = umap_model.fit_transform(
    cfret_screen_df[cfret_screen_feats].to_numpy()
)

# concatenate UMAP embeddings with metadata
umap_df = pl.DataFrame(umap_embeddings, schema=["UMAP_1", "UMAP_2"])
cfret_umap_df = cfret_screen_df[cfret_screen_meta].hstack(umap_df)

# save cfret_umap_df
cfret_umap_df.write_parquet(
    (results_dir / "cfret_screen_treatment_umap.parquet").resolve(strict=False)
)

# display
cfret_umap_df.head()